# TP 3 - Opérations locales

Dans ce TP, nous allons traiter les images à une échelle locale. Il s'agit de considérer l'image par "régions".

Ce TP est **noté**. Vous pouvez rendre votre code en m'envoyant le fichier Jupyter (format `ipynb`) par mail à `florent.grelard [at] u-bordeaux.fr`.

Vous répondrez aux questions "ouvertes" ("Que constatez-vous...?", "Pourquoi...?", etc.) dans le TP par des commentaires dans votre code Python. Toutes tentatives et justifications pertinentes seront valorisées.

Vous pouvez vous aider de toutes les fonctions numpy/skimage existantes. Si vous êtes bloqués à une question, vous pouvez utiliser les fonctions de ces bibliothèques pour passer aux questions suivantes.

## 1. Opérateurs de morphologie mathématique

Dans ce premier exercice, l'objectif est d'extraire une image de contours en utilisant les opérateurs de morphologie mathématique. Dans un premier temps, nous allons implémenter les opérateurs d'**érosion** et de **dilatation**. L'image de contours sera donnée par le **gradient morphologique**, défini par $I_{\text{dilatée}} - I_{\text{érodée}}$.

<table>
    <tr>
    <td>
        <img src="data/obj_morphomaths.png" width="700" /> 
        <figcaption> De gauche à droite: (a) Image originale, (b) Image érodée avec une taille de 10, (c) Image dilatée avec une taille de 10, (d) Image de gradient avec une taille de 1 </figcaption>
    </td>
    </tr>
</table>

Les opérateurs de morphologie mathématique classiques fonctionnent sur une image binaire. Une région, appelée *élément structurant*, est positionnée en chaque pixel et détermine la nouvelle appartenance à l'objet ou à l'arrière-plan en fonction de l'opérateur:
* pour l'**érosion**: pour qu'un pixel soit objet, tous les pixels dans la région doivent être objet.
* pour la **dilatation**: pour qu'un pixel soit objet, au moins un pixel dans la région doit être objet.

**Exercice**: Compléter le code suivant:
1. Compléter la fonction `extract_region` qui extrait la sous-image à la position (x,y) pour une région de rayon `size`.
2. Compléter les fonctions `erosion` et `dilatation`. Afficher les images obtenues. 
3. Utiliser une taille d'élement structurant égale à 10. Que constatez-vous? 
4. Résoudre le problème précédent en employant la méthode de votre choix. Vous pouvez vous aider des supports de cours ainsi que de la documentation numpy. Préciser la méthode employée en commentaire.
5. Calculer le gradient morphologique (fonction `morphological_gradient`) et afficher le résultat.
6. Comparer vos résultats avec ceux obtenus par les fonctions de la bibliothèque `skimage.morphology`.

In [ ]:
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage import color

def extract_region(image, x, y, size):
    """
    Extrait la sous-matrice à la position (x,y)
    pour une région de taille `size`
    """
    return image[x-size: x+size+1, y-size: y+size+1]

def erosion(image, size):
    image_copy = np.pad(image, size)
    image_out = image_copy.copy()
    for x,y in np.ndindex(image.shape):
        x_n = x+size
        y_n = y+size
        region = extract_region(image_copy, x_n, y_n, size)
        if region.all():
            image_out[x_n, y_n] = 255
        else:
            image_out[x_n, y_n] = 0
    image_out = image_out[size:-size, size:-size]
    return image_out

def dilatation(image, size):
    image_copy = np.pad(image, size)
    image_out = image_copy.copy()
    for x,y in np.ndindex(image.shape):
        x_n = x+size
        y_n = y+size
        region = extract_region(image_copy, x_n, y_n, size)
        if region.any():
            image_out[x_n, y_n] = 255
        else:
            image_out[x_n, y_n] = 0
    image_out = image_out[size:-size, size:-size]
    return image_out

def morphological_gradient(image, size):
    eroded = erosion(image, size)
    dilated = dilatation(image, size)
    return dilated - eroded

img_disk = io.imread("data/disquette.jpg")

# On seuille pour obtenir une image binaire
img_disk = np.where(img_disk > 127, 0, 255).astype(np.uint8)

size = 1

img_eroded = erosion(img_disk, 10)
img_dilation = dilatation(img_disk, 10)
img_gradient = morphological_gradient(img_disk, size)
img_diff = img_disk - img_eroded
fig, ax = plt.subplots(1, 4)

ax[0].imshow(img_disk, cmap="gray")
ax[1].imshow(img_eroded, cmap="gray")
ax[2].imshow(img_dilation, cmap="gray")
ax[3].imshow(img_gradient, cmap="gray")
[axi.set_axis_off() for axi in ax]

## 2. Filtres et convolution

Les filtres sont des régions carrées centrées en un pixel avec des coefficients variables afin, par exemple, de flouter l'image, de réduire le bruit, ou encore d'extraire le contour des objets. 

La convolution consiste en la somme des produits des coefficients du filtre $F$ avec les intensités originales de l'image $I$ dans un voisinage $V$ : 

$
    I'(x, y) = \sum_{(i, j) \in V(x, y)} I(i, j) * F(i, j)
$

Nous cherchons à débruiter l'image afin d'obtenir le résultat ci-dessous:

<table>
<tr>
<td>
<img src="data/obj_bruit.png" width="500"> 
<figcaption style="text-align:center;"> De gauche à droite: (a) Image originale, (b) Image débruitée </figcaption>
</td>
</tr>
</table>

Nous allons implémenter le filtre moyen, ainsi que le filtre médian.

**Exercice**: Compléter le code suivant:
1. Compléter la fonction `mean_filter_coefficients` pour générer une région de taille $n \times n$ avec les coefficients du filtre moyen. On rappelle que les coefficients du filtre moyen de taille $n \times n$ sont tous égaux à $\frac{1}{n^2}$.
2. Compléter la fonction `convolution` qui permet de calculer une nouvelle intensité par convolution d'une région de l'image et d'un filtre de taille $n \times n$.
3. Compléter la fonction `generic_filtering` qui permet de calculer une image filtrée par convolution avec un filtre quelconque. Cette fonction appelle la fonction `convolution` sur l'ensemble de l'image.
4. Utiliser la fonction `generic_filtering` pour appliquer un filtrage moyen.
5. Compléter la fonction `median_filtering` pour calculer le filtrage médian. Cette fonction n'utilise pas la fonction `convolution` puisqu'elle n'utilise pas de filtre avec des coefficients.
6. Comparer les résultats obtenus avec les filtres moyen et médian. Quel filtre semble plus adapté à réduire le bruit? Pourquoi?
7. Comparer vos résultats avec ceux obtenus par les fonctions du module `skimage.filters.rank`.

In [ ]:
def mean_filter_coefficients(size):
    """
    Filtre de taille size x size
    """
    return np.ones((size, size))/size**2

def convolution(region, filtre):
    """
    Convolution d'une région de l'image
    avec un filtre

    Prérequis: `region` et `filtre` doivent être de la même taille


    Parameters
    ----------
    region: np.ndarray
        nxn region
    filtre: np.ndarray
        nxn filter with coefficients

    Returns
    -------
    new_intensity: float
        la nouvelle intensité obtenue par convolution
    """
    return np.sum(region * filtre)

def generic_filtering(image, filtre):
    """
    Convolution avec filtre moyen

    Parameters
    ----------
    image: np.ndarray
        image
    filtre: np.ndarray
        filtre de taille nxn

    Returns
    -------
    image: np.ndarray
        image filtrée
    """
    size = filtre.shape[0]
    radius = size//2
    image_copy = np.pad(image, radius)
    image_out = image_copy.copy()
    for (x, y) in np.ndindex(image.shape):
        x += radius
        y += radius
        region = extract_region(image_copy, x, y, radius)
        new_intensity = convolution(region, filtre)
        image_out[x, y] = new_intensity
    return image_out[radius:-radius, radius:-radius]

def median_filtering(image, size):
    """
    Application du filtrage médian

    Parameters
    ----------
    image: np.ndarray
        image
    size: int
        taille de la région (size * size)

    Returns
    -------
    image: np.ndarray
        image filtrée
    """
    radius = size//2
    image_copy = np.pad(image, radius)
    image_out = image_copy.copy()
    for (x, y) in np.ndindex(image.shape):
        x += radius
        y += radius
        region = extract_region(image, x, y, radius)
        image_out[x, y] = np.median(region)
    return image_out[radius:-radius, radius:-radius]

img_noisy = io.imread("data/bruit.jpg")
mean_coeffs = mean_filter_coefficients(5)
img_mean = generic_filtering(img_noisy, mean_coeffs)
img_median = median_filtering(img_noisy, 5)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(img_noisy, cmap="gray")
ax[1].imshow(img_median, cmap="gray")
[axi.set_axis_off() for axi in ax]